In [1]:
from keras.applications import MobileNet

Using TensorFlow backend.


In [2]:
model = MobileNet(weights='imagenet' , include_top=False , input_shape=(224,224,3) )

In [3]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
model.layers[0].trainable

False

In [5]:
model.output

<tf.Tensor 'conv_pw_13_relu/Relu6:0' shape=(None, 7, 7, 1024) dtype=float32>

In [6]:
for i in model.layers:
    i.trainable=False

In [7]:
model.layers[6].trainable

False

In [8]:
top_model = model.output

In [9]:
from keras.layers import Dense

In [10]:
from keras.models import Sequential

In [11]:
from keras.layers import GlobalAveragePooling2D

In [12]:
from keras.layers import Flatten

In [13]:
top_model = Flatten()(top_model)

In [14]:
top_model = Dense(256 , activation='relu' )(top_model)

In [15]:
top_model = Dense(128 , activation='relu' )(top_model)

In [16]:
top_model = Dense(2 , activation='softmax' )(top_model)

In [17]:
from keras.models import Model

In [18]:
model= Model(inputs=model.input , outputs = top_model)

In [19]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [20]:
from keras.optimizers import Adam

In [21]:
model.compile(optimizer=Adam(learning_rate=0.01) , loss='binary_crossentropy' , metrics=['accuracy'])

In [22]:
from keras.preprocessing.image import ImageDataGenerator

In [23]:
traingen = ImageDataGenerator(rescale=1./255 , zoom_range=0.2 , horizontal_flip=True ,  vertical_flip=True )

In [24]:
testgen = ImageDataGenerator(rescale=1./255)

In [25]:
trainset = traingen.flow_from_directory("C:/Users/ManishVerma/Desktop/dataset/train" , target_size=(224,224) , batch_size=32 , class_mode='categorical' )

Found 694 images belonging to 2 classes.


In [26]:
testset = testgen.flow_from_directory("C:/Users/ManishVerma/Desktop/dataset/validation/" , target_size=(224,224) , batch_size=32 , class_mode='categorical' )

Found 200 images belonging to 2 classes.


In [ ]:
history=model.fit(trainset , epochs=5 , validation_data=testset , steps_per_epoch=10, validation_steps=10  )
model.save("transfer_learning.h5")
score=model.evaluate(testset,verbose=1)
print('Test loss:',score[2])
print('test accuracy:', score[3])

Epoch 1/5
10/10 [==============================] - 62s 6s/step - loss: 7.0355 - accuracy: 0.4903 - val_loss: 5.2721 - val_accuracy: 0.5101
Epoch 2/5
10/10 [==============================] - 63s 6s/step - loss: 7.8818 - accuracy: 0.4875 - val_loss: 6.7100 - val_accuracy: 0.4831
Epoch 3/5
10/10 [==============================] - 64s 6s/step - loss: 8.0667 - accuracy: 0.4774 - val_loss: 5.7514 - val_accuracy: 0.5037
Epoch 4/5
 5/10 [==============>...............] - ETA: 16s - loss: 7.8818 - accuracy: 0.4875

In [ ]:
from keras.models import load_model

classifier = load_model("transfer_learning.h5")

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

model_dict = {"[0]": "alien", 
                      "[1]": "predator",
                      }

model_dict_n = {"alien": "Alien", 
                      "predator": "Predator",

                      }0


def draw_test(name, pred, im):
    monkey = model_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + model_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("C:/Users/ManishVerma/Desktop/dataset/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
model_dict = {"[0]": "Ben Afflek", 
                      "[1]": "Elton John",
                      "[2]": "Jerry Seinfeld",
                      "[3]": "Madonna",
                      "[4]": "Mindy Kaling",
                      }

model_dict_n = {"ben_afflek": "Ben Afflek", 
                      "elton_john": "Elton John",
                      "jerry_seinfeld": "Jerry Seinfeld",
                      "madonna": "Madonna",
                      "mindy_kaling": "Mindy Kaling",
                      }0
